In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [2]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [3]:
def clean_figure(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis="x", rotation=45)

In [17]:
f = 'varSpecPopInTrSpecSamePop_changeConseq.txt'
df = pd.read_csv(f, sep='\t')

In [19]:
# i want variants that affect the protein
print(df['poder.cons'].unique())

vep_cats = ['missense_variant', 'start_lost']
df = df.loc[df['poder.cons'].isin(vep_cats)]

['intron_variant' 'non_coding_transcript_exon_variant'
 '5_prime_UTR_variant' 'missense_variant' '3_prime_UTR_variant'
 'splice_donor_region_variant' 'splice_polypyrimidine_tract_variant'
 'splice_region_variant' 'splice_donor_variant' 'start_lost'
 'splice_acceptor_variant']


In [20]:
df

,seqname.tr,start.tr,end.tr,tr_id.tr,score.tr,strand.tr,seqname.var,start.var,end.var,all.var,uploadedVar.var,gencode.cons,enhanced.cons,poder.cons,var_pop,tr_pop
13,chr12,4649095,4687240,transcript_39863,0,+,chr12,4662515,4662516,A,12:4662516:G:A,splice_polypyrimidine_tract_variant,missense_variant,missense_variant,PEL,PEL
14,chr14,24136203,24138962,transcript_52311,0,+,chr14,24137242,24137243,G,14:24137243:A:G,non_coding_transcript_exon_variant,missense_variant,missense_variant,PEL,PEL
27,chr17,7012609,7017525,transcript_77468,0,+,chr17,7016316,7016317,T,17:7016317:C:T,non_coding_transcript_exon_variant,5_prime_UTR_variant,missense_variant,PEL,PEL
28,chr17,7014798,7017519,transcript_77550,0,+,chr17,7016316,7016317,T,17:7016317:C:T,non_coding_transcript_exon_variant,5_prime_UTR_variant,missense_variant,PEL,PEL
29,chr17,7015036,7017522,transcript_77591,0,+,chr17,7016316,7016317,T,17:7016317:C:T,non_coding_transcript_exon_variant,5_prime_UTR_variant,missense_variant,PEL,PEL
30,chr17,7015039,7017523,transcript_77599,0,+,chr17,7016316,7016317,T,17:7016317:C:T,non_coding_transcript_exon_variant,5_prime_UTR_variant,missense_variant,PEL,PEL
38,chr20,3209278,3223870,transcript_112215,0,+,chr20,3221422,3221423,T,20:3221423:C:T,intron_variant,missense_variant,missense_variant,ITU,ITU
39,chr20,3209278,3223870,transcript_112215,0,+,chr20,3223437,3223438,A,20:3223438:G:A,synonymous_variant,missense_variant,missense_variant,ITU,ITU
118,chr3,189309649,189321060,transcript_132268,0,+,chr3,189309926,189309927,G,3:189309927:A:G,non_coding_transcript_exon_variant,missense_variant,start_lost,PEL,PEL
244,chr6,32637401,32746416,transcript_158783,0,+,chr6,32659936,32659937,G,6:32659937:T:G,3_prime_UTR_variant,missense_variant,missense_variant,PEL,PEL


In [9]:
# [c for c in df['poder.cons'].unique() if 'splice' in c]

# i want variants that affect the splicing 
vep_cats = ['splice_donor_region_variant', 'splice_region_variant', 'splice_donor_variant',
            'splice_acceptor_variant']
df = df.loc[df['poder.cons'].isin(vep_cats)]

In [12]:
df

,seqname.tr,start.tr,end.tr,tr_id.tr,score.tr,strand.tr,seqname.var,start.var,end.var,all.var,uploadedVar.var,gencode.cons,enhanced.cons,poder.cons,var_pop,tr_pop
21,chr15,64155811,64163035,transcript_254743,0,-,chr15,64162923,64162924,T,15:64162924:G:T,synonymous_variant,splice_donor_region_variant,splice_donor_region_variant,PEL,PEL
26,chr17,75260938,75266381,transcript_78438,0,+,chr17,75261724,75261725,A,17:75261725:G:A,5_prime_UTR_variant,splice_region_variant,splice_region_variant,PEL,PEL
33,chr17,55431660,55437134,transcript_269979,0,-,chr17,55433126,55433127,A,17:55433127:G:A,intergenic_variant,splice_donor_variant,splice_donor_variant,PEL,PEL
248,chr6,32637401,32746416,transcript_158783,0,+,chr6,32662177,32662178,T,6:32662178:C:T,synonymous_variant,splice_region_variant,splice_region_variant,PEL,PEL
270,chr6,32637401,32746416,transcript_158783,0,+,chr6,32667488,32667489,G,6:32667489:T:G,intron_variant,splice_region_variant,splice_region_variant,PEL,PEL
422,chr6,32637397,32746898,transcript_158784,0,+,chr6,32662177,32662178,T,6:32662178:C:T,synonymous_variant,splice_region_variant,splice_region_variant,PEL,PEL
444,chr6,32637397,32746898,transcript_158784,0,+,chr6,32667488,32667489,G,6:32667489:T:G,intron_variant,splice_region_variant,splice_region_variant,PEL,PEL
492,chr6,33065013,33073656,transcript_347504,0,-,chr6,33070213,33070214,G,6:33070214:C:G,intron_variant,splice_donor_variant,splice_donor_variant,YRI,YRI
495,chr6,33065013,33073656,transcript_347504,0,-,chr6,33070275,33070276,T,6:33070276:C:T,intron_variant,splice_acceptor_variant,splice_acceptor_variant,YRI,YRI
500,chr6,33065013,33073656,transcript_347504,0,-,chr6,33071828,33071829,T,6:33071829:C:T,intron_variant,splice_region_variant,splice_region_variant,YRI,YRI
